# 05 Correct External Stations Centroid Connector Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks - Copy')
root_dir = "C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_04')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_06')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "C:/Users/USGS667806/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-09 07:43:04, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 07:43:04, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 07:43:04, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 07:43:04, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso


## Read Version 03 Scenario

In [6]:
link_file = os.path.join(input_dir, 'v02_link.json')
node_file = os.path.join(input_dir, 'v02_node.geojson')
shape_file = os.path.join(input_dir, 'v02_shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

2021-02-09 07:43:05, INFO: Reading RoadwayNetwork
2021-02-09 07:43:05, INFO: Reading RoadwayNetwork
2021-02-09 07:47:38, INFO: Read 2229982 links from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04\v02_link.json
2021-02-09 07:47:38, INFO: Read 2229982 links from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04\v02_link.json
2021-02-09 07:47:38, INFO: Read 696439 nodes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04\v02_node.geojson
2021-02-09 07:47:38, INFO: Read 696439 nodes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04\v02_node.geojson
2021-02-09 07:47:38, INFO: Read 1169752 shapes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04\v02_shape.geojson
2021-02-09 07:47:38, INFO: Read 1169752 shapes from C:/Users/USGS667806/Documents/GitH

In [7]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-02-09 07:53:35, INFO: Read in transit feed from: C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04
2021-02-09 07:53:35, INFO: Read in transit feed from: C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_04
2021-02-09 07:53:35, INFO: Removing calendar.txt from transit network config because file not found
2021-02-09 07:53:35, INFO: Removing calendar.txt from transit network config because file not found
2021-02-09 07:53:35, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-09 07:53:35, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-09 07:53:37, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-02-09 07:53:37, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-02-09 07:53:37, INFO: Removing fare_rules.txt from transit network co

In [11]:
base_scenario = {"road_net": roadway_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2021-02-09 07:56:20, INFO: Creating Scenario
2021-02-09 07:56:20, INFO: Creating Scenario


In [12]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_04.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))

In [13]:
version_04_pickle_file_name = os.path.join(output_dir, 'working_scenario_04.pickle')
v_04_scenario = pickle.load(open(version_04_pickle_file_name, 'rb'))

In [14]:
v_04_scenario.road_net.links_df.shape

(2229982, 56)

In [15]:
v_04_scenario.transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   route_id_original       701 non-null    object
 1   agency_id               526 non-null    object
 2   route_short_name        665 non-null    object
 3   route_long_name         679 non-null    object
 4   route_desc              14 non-null     object
 5   route_type              701 non-null    int64 
 6   route_url               217 non-null    object
 7   route_color             260 non-null    object
 8   route_text_color        243 non-null    object
 9   agency_raw_name         701 non-null    object
 10  route_sort_order        95 non-null     object
 11  min_headway_minutes     32 non-null     object
 12  eligibility_restricted  4 non-null      object
 13  continuous_pickup       4 non-null      object
 14  continuous_drop_off     4 non-null      object
 15  route_

## Create Scenario and Apply Project Cards

In [16]:
v_05_scenario = Scenario.create_scenario(
    base_scenario=v_04_scenario,
    card_directory = card_dir,
    tags = ['External Stations Review'],
    validate_project_cards=False)

2021-02-09 08:00:31, INFO: Creating Scenario
2021-02-09 08:00:31, INFO: Creating Scenario


In [17]:
v_05_scenario.apply_all_projects()

2021-02-09 08:00:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:00:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:00:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:00:47, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:00:47, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:00:47, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-02-09 08:02:18, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Fr

2021-02-09 08:15:23, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Santa Cruz Highway CA-17 NB
2021-02-09 08:16:56, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:16:56, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:16:56, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:16:56, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:16:56, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:16:56, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 SB
2021-02-09 08:18:31, INFO: Apply

2021-02-09 08:33:23, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 WB
2021-02-09 08:33:23, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 WB
2021-02-09 08:33:23, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 WB
2021-02-09 08:33:23, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 WB
2021-02-09 08:33:23, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 WB
2021-02-09 08:35:02, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-02-09 08:35:02, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway 

In [18]:
v_05_scenario.applied_projects

['Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB',
 'Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB',
 'Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-113 NB',
 'Year 2015 Add centroid connector at External Station connecting Vic Fazio Freeway CA-113 SB',
 'Year 2015 Add centroid connector at External Station connecting South Valley Freeway US-101 SB',
 'Year 2015 Add centroid connector at External Station connecting South Valley Freeway US-101 NB',
 'Year 2015 Add centroid connector at External Station connecting Skyline Boulevard CA-35 SB',
 'Year 2015 Add centroid connector at External Station connecting Skyline Boulevard CA-35 NB',
 'Year 2015 Add centroid connector at External Station connecting Santa Cruz Highway CA-17 SB',
 'Year 2015 Add centroid connector at External Station connecting Santa Cruz Highway CA-17 NB',
 'Year 2015 Add ce

In [19]:
v_05_scenario.road_net.links_df.columns

Index(['index', 'access', 'bike_access', 'drive_access', 'fromIntersectionId',
       'lanes', 'maxspeed', 'name', 'oneWay', 'ref', 'roadway',
       'shstGeometryId', 'shstReferenceId', 'toIntersectionId', 'u', 'v',
       'walk_access', 'wayId', 'county', 'model_link_id', 'A', 'B',
       'rail_traveltime', 'rail_only', 'locationReferences', 'shape_id',
       'ft_cal', 'ft', 'useclass', 'assignable', 'transit', 'managed',
       'segment_id', 'tollseg', 'bus_only', 'transit_access', 'egress', 'id',
       'distance', 'cntype', 'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM',
       'lanes_EV', 'ML_lanes_EA', 'ML_lanes_AM', 'ML_lanes_MD', 'ML_lanes_PM',
       'ML_lanes_EV', 'useclass_EA', 'useclass_AM', 'useclass_MD',
       'useclass_PM', 'useclass_EV', 'geometry'],
      dtype='object')

## Make Travel Model Networks

In [20]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_05_scenario.road_net, 
    parameters = parameters)

2021-02-09 08:53:18, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:53:18, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:53:18, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:53:18, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso


In [21]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2230008, 56)
(696439, 12)
(1169778, 8)


In [22]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2230008, 56)
(696439, 12)
(1169778, 8)


In [23]:
model_net.links_df.model_link_id.nunique()

2230008

In [25]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2021-02-09 08:59:07, INFO: Adding tap and tap connector to standard network
2021-02-09 08:59:07, INFO: Adding tap and tap connector to standard network
2021-02-09 08:59:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:59:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:59:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:59:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 08:59:15, INFO: Finished adding tap and tap connectors
2021-02-09 08:59:15, INFO: Finished adding tap and tap connectors


In [26]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2300492, 56)
(708439, 12)
(1205020, 8)


In [27]:
model_net.links_df.model_link_id.nunique()

2230008

In [28]:
model_net.nodes_df.model_node_id.nunique()

696439

In [29]:
model_net.shapes_df.id.nunique()

1169778

In [30]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-02-09 09:00:11, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-09 09:00:11, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-09 09:00:11, INFO: Creating managed lane network.
2021-02-09 09:00:11, INFO: Creating managed lane network.
2021-02-09 09:00:11, INFO: Creating network with duplicated managed lanes
2021-02-09 09:00:11, INFO: Creating network with duplicated managed lanes
2021-02-09 09:00:11, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2021-02-09 09:00:11, ERROR: managed lane access links already exist in network; shouldn't be running create managed lane network. Returning network as-is.
2021-02-09 09:00:11, INFO: Determining cntype
2021-02-09 09:00:11, INFO: Determining cntype
2021-02-09 09:00:11, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-09 09:00:11, INFO: Lasso base 

ValueError: could not convert string to float: 

In [ ]:
model_net.links_df.shape

In [ ]:
model_net.links_df.model_link_id.nunique()

In [ ]:
model_net.nodes_mtc_df.shape

In [ ]:
model_net.nodes_df.model_node_id.nunique()

## Write to Disk

## As shapefile

In [ ]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

## As fixed width (for CUBE)

In [ ]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

In [ ]:
%%capture
os.chdir(output_dir)
!make_cube.bat

In [ ]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s
!runtpp make_highway_ext_stn_connector_network.s

## As transit line file (for CUBE)

In [ ]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_05_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

## As pickle

In [ ]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_04.pickle')
pickle.dump(v_04_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [ ]:
model_net.write(path = output_dir, filename = "v04")